<a href="https://colab.research.google.com/github/mabdulrokhim/Portfolio-DataScience/blob/main/Flight_Fare_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal :** *Memprediksi harga tiket pesawat terbang menggunakan ML berdasarkan tanggal, tempat, airline, dan stop.*

**What I Learn :**


1.   Feature Engineering : Categorical and Time
2.   Deployment in Flask and Streamlit



## Download Data

Pertama, import beberapa modul umum seperti pandas, matplotlib dan numpy. Selanjutnya pastikan MatplotLib untuk memplot secara inline dan menyiapkan fungsi untuk menyimpan plot.

In [ ]:
# setup library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#setup pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.precision', 3)

#setup matplotlib
import matplotlib as mpl
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# mount google colab utk setup kaggle
from google.colab import drive
drive.mount('/content/drive')

# install kaggle
!pip install kaggle

#setup authentication API
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Colab Notebooks/KAGGLE/"

# get dataset via copy command api
!kaggle datasets download -d nikhilmittal/flight-fare-prediction-mh

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/626k [00:00<?, ?B/s]
100% 626k/626k [00:00<00:00, 123MB/s]


In [ ]:
#unzip data
!unzip flight-fare-prediction-mh.zip -d "/content/drive/MyDrive/Colab Notebooks/DATASET"

Archive:  flight-fare-prediction-mh.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/DATASET/Data_Train.xlsx  
  inflating: /content/drive/MyDrive/Colab Notebooks/DATASET/Sample_submission.xlsx  
  inflating: /content/drive/MyDrive/Colab Notebooks/DATASET/Test_set.xlsx  


In [ ]:
PATH_TRAIN = "/content/drive/MyDrive/Colab Notebooks/DATASET/Data_Train.xlsx"
PATH_TEST = "/content/drive/MyDrive/Colab Notebooks/DATASET/Sample_submission.xlsx"
PATH_SUB = "/content/drive/MyDrive/Colab Notebooks/DATASET/Test_set.xlsx"

In [ ]:
train = pd.read_excel(PATH_TRAIN)
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


Data type semuanya object nanti perlu diubah

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [ ]:
train.isnull().sum()

Airline            0
Source             0
Destination        0
Route              1
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
isWeekend          0
Journey_day        0
Journey_month      0
depHour            0
depMin             0
dtype: int64

Explore Each column

In [ ]:
train['Duration'].value_counts()

2h 50m     550
1h 30m     386
2h 45m     337
2h 55m     337
2h 35m     329
          ... 
31h 30m      1
30h 25m      1
42h 5m       1
4h 10m       1
47h 40m      1
Name: Duration, Length: 368, dtype: int64

In [ ]:
train['Date_of_Journey'].value_counts()

18/05/2019    504
6/06/2019     503
21/05/2019    497
9/06/2019     495
12/06/2019    493
9/05/2019     484
21/03/2019    423
15/05/2019    405
27/05/2019    382
27/06/2019    355
24/06/2019    351
1/06/2019     342
3/06/2019     333
15/06/2019    328
24/03/2019    323
6/03/2019     308
27/03/2019    299
24/05/2019    286
6/05/2019     281
1/05/2019     277
12/05/2019    259
1/04/2019     257
3/03/2019     218
9/03/2019     200
15/03/2019    162
18/03/2019    156
01/03/2019    152
12/03/2019    142
9/04/2019     125
3/04/2019     110
21/06/2019    109
18/06/2019    105
09/03/2019    102
6/04/2019     100
03/03/2019     97
06/03/2019     95
27/04/2019     94
24/04/2019     92
3/05/2019      90
15/04/2019     89
21/04/2019     82
18/04/2019     67
12/04/2019     63
1/03/2019      47
Name: Date_of_Journey, dtype: int64

In [ ]:
pd.to_datetime(train['Date_of_Journey'], format="%d/%m/%Y").dt.day_name()

0           Sunday
1        Wednesday
2           Sunday
3           Sunday
4           Friday
           ...    
10678      Tuesday
10679     Saturday
10680     Saturday
10681       Friday
10682     Thursday
Name: Date_of_Journey, Length: 10682, dtype: object

In [ ]:
# dateJourney = pd.read_excel(PATH_TRAIN,usecols=["Date_of_Journey","Dep_Time"])
trainBefore = pd.read_excel(PATH_TRAIN)
trainBefore.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [ ]:
pd.to_datetime(train['Date_of_Journey'], format="%d/%m/%Y").dt.year.value_counts()

2019    10683
Name: Date_of_Journey, dtype: int64

In [ ]:
dayWeek = pd.to_datetime(train['Date_of_Journey'], format="%d/%m/%Y").dt.dayofweek
train['isWeekend'] = dayWeek.apply(lambda x: 1 if x in [5,6] else 0)
# train.head()

In [ ]:
#Get Journey Day
#ubah object ke datetime64 type
#.dt punya banyak opsi utk akses data - lihat di doc cari .dt.
#tahunnya cuman ada di 2019

train["Journey_day"] = pd.to_datetime(train['Date_of_Journey'], format="%d/%m/%Y").dt.day
train["Journey_month"] = pd.to_datetime(train['Date_of_Journey'], format="%d/%m/%Y").dt.month

#drop column
train.drop(columns=['Date_of_Journey'], inplace=True)
# train.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,1,24,3
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,0,1,5
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,1,9,6
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,1,12,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,0,1,3


In [ ]:
# Extracting Departure Time (Leaving Gate)
train["depHour"] = pd.to_datetime(train['Dep_Time']).dt.hour
train["depMin"] = pd.to_datetime(train['Dep_Time']).dt.minute

#drop Dep_Time
train.drop(columns="Dep_Time", inplace=True)
# train.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,1,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,0,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,1,9,6,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218,1,12,5,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302,0,1,3,16,50


In [ ]:
# Extracting Arrival Time
train["arrHour"] = pd.to_datetime(train['Arrival_Time']).dt.hour
train["arrMin"] = pd.to_datetime(train['Arrival_Time']).dt.minute

#drop Dep_Time
train.drop(columns="Arrival_Time", inplace=True)
# train.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin,arrHour,arrMin
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,1,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,0,1,5,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,1,9,6,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,1,12,5,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,0,1,3,16,50,21,35


In [ ]:
#Extracting Duration pakai functional programming
train["durationHour"]=train["Duration"].apply(lambda x: x.split(" ")[0].replace('h','') if "h" in x.split(" ")[0] else "0")
train["durationMin"]=train["Duration"].apply(lambda x: x.split()[1].replace("m","") if (len(x.split(" "))==2) else (x.split()[0].replace("m","") if ("m" in (x.split(" ")[0])) else "0"))

#untuk cek ganti aja 0 dengan 0m atau 0h, kemudian cari pakai .loc
# train.loc[train['durationMin']=="0m",['Duration',"durationHour","durationMin"]]

#drop duration
train.drop(columns="Duration", inplace=True)

In [ ]:
#value yang tidak masuk akal
train.loc[train["durationHour"]=="0"]

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin,arrHour,arrMin,durationHour,durationMin
6474,Air India,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,2 stops,No info,17327,0,6,3,16,50,16,55,0,5


In [ ]:
train.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin,arrHour,arrMin,durationHour,durationMin
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,1,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,0,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,1,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,1,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,0,1,3,16,50,21,35,4,45


CATEGORICAL -> PERLU EKSPLOR PAKE VISUALISASI DATA


*  Categorical Variabel lebih butuh banyak dieksplor dibandingkan Numerical
*  Bagusnya sejak awal dibedakan dulu waktu eksplorasi, krn notebook ini kecampur antara EDA dan FE



In [ ]:
#sns catplot price airline
#one hot encoding utk airline, gak perlu target encoding
#sns catplot utk source
#one hot encoding utk source
#sns catplot utk destination
#one hot encoding utk destination

https://youtu.be/y4EMEpEnElQ?t=4234
#drop route dan additional info
#label encoder utk Total_Stops
#drop kolom yg sudah di-encode

Other

In [ ]:
train=train[['Airline', 'Source', 'Destination', 'Route', 'Arrival_Time', 'Duration',
       'Total_Stops', 'Additional_Info', 'Price', 'isWeekend', 'Journey_day',
       'Journey_month', 'depHour', 'depMin', 'Dep_Time']]
train.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin,Dep_Time
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,1,24,3,22,20,22:20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,0,1,5,5,50,05:50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,1,9,6,9,25,09:25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218,1,12,5,18,5,18:05
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302,0,1,3,16,50,16:50


In [ ]:
#Cari tahu value tanggal di Arrival Time
#Kenapa banyak yg kosong bernilai dua
train["dayArr"] = pd.to_datetime(train['Arrival_Time']).dt.day
train["Date_of_Journey"] = dateJourney["Date_of_Journey"]
train["Dep_Time"] = dateJourney["Dep_Time"]
train["dayArr"].value_counts()

2     6804
10     621
7      588
13     465
28     429
25     409
22     403
16     372
19     305
4      236
23      13
29      11
8        8
11       5
26       5
20       4
3        2
5        2
17       1
Name: dayArr, dtype: int64

In [ ]:
#day arrival kacau, durasi masih oke
train[["Date_of_Journey","Dep_Time","Arrival_Time", "Duration","dayArr"]].loc[train["dayArr"]==2]

,Date_of_Journey,Dep_Time,Arrival_Time,Duration,dayArr
1,1/05/2019,05:50,13:15,7h 25m,2
3,12/05/2019,18:05,23:30,5h 25m,2
4,01/03/2019,16:50,21:35,4h 45m,2
5,24/06/2019,09:00,11:25,2h 25m,2
7,01/03/2019,08:00,05:05 02 Mar,21h 5m,2
...,...,...,...,...,...
10678,9/04/2019,19:55,22:25,2h 30m,2
10679,27/04/2019,20:45,23:20,2h 35m,2
10680,27/04/2019,08:20,11:20,3h,2
10681,01/03/2019,11:30,14:10,2h 40m,2


In [ ]:
train.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,isWeekend,Journey_day,Journey_month,depHour,depMin
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,1,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,0,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,1,9,6,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218,1,12,5,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302,0,1,3,16,50


Feature Engineering

## Building Regression Pipeline

In [ ]:
# Test Model
from sklearn.metrics import mean_squared_error
final_model = grid_search.best_estimator_

X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

48878.755198880746

In [ ]:
# Hasil Best Score utk Grid Search
np.sqrt(-grid_search.best_score_)

48753.497098183034

In [ ]:
100*(final_rmse-np.sqrt(-grid_search.best_score_))/final_rmse

0.2562628696006169

Kesimpulannya hasil pengujian model di test data itu sedikit overfit sebesar 0.256%

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=150, random_state=42)

In [ ]:
housing

,longitude,latitude,housing_median_age (per_block),total_rooms (per_block),total_bedrooms (per_block),population (per_block),households (per_block),median_income (per house),median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.325,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.301,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.257,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.643,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.846,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.560,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.557,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.700,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.867,84700.0,INLAND


In [ ]:
housing_ = housing.copy()
housing_

,longitude,latitude,housing_median_age (per_block),total_rooms (per_block),total_bedrooms (per_block),population (per_block),households (per_block),median_income (per house),median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.325,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.301,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.257,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.643,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.846,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.560,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.557,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.700,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.867,84700.0,INLAND


In [ ]:
housing_labels = housing_['median_house_value']
housing_ = housing_.drop(columns = 'median_house_value')

housing_labels

0        452600.0
1        358500.0
2        352100.0
3        341300.0
4        342200.0
           ...   
20635     78100.0
20636     77100.0
20637     92300.0
20638     84700.0
20639     89400.0
Name: median_house_value, Length: 20640, dtype: float64

In [ ]:
some_data = housing_.sample(n=10)
some_data

,longitude,latitude,housing_median_age (per_block),total_rooms (per_block),total_bedrooms (per_block),population (per_block),households (per_block),median_income (per house),ocean_proximity
5939,-117.90,34.14,35.0,2259.0,505.0,1561.0,509.0,3.304,<1H OCEAN
2203,-119.85,36.82,9.0,3995.0,778.0,1691.0,712.0,3.324,INLAND
16297,-121.22,37.96,30.0,1737.0,381.0,1177.0,347.0,1.988,INLAND
4653,-118.33,34.06,52.0,1368.0,231.0,737.0,248.0,8.362,<1H OCEAN
1813,-122.31,37.91,39.0,2955.0,696.0,1417.0,682.0,2.763,NEAR BAY
4989,-118.31,34.00,47.0,1551.0,362.0,1329.0,322.0,1.979,<1H OCEAN
11277,-117.96,33.80,33.0,1984.0,420.0,1119.0,387.0,3.482,<1H OCEAN
16271,-121.25,37.94,30.0,1509.0,308.0,967.0,278.0,1.780,INLAND
13964,-116.86,34.24,19.0,5411.0,1042.0,441.0,185.0,3.132,INLAND
12180,-117.31,33.75,19.0,3173.0,678.0,2204.0,606.0,2.148,<1H OCEAN


In [ ]:
# full pipeline
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("best", RandomForestRegressor(max_features=6, n_estimators=150, random_state=42))
    ])

full_pipeline_with_predictor.fit(housing_, housing_labels)
pred = full_pipeline_with_predictor.predict(some_data)
pred

array([166840.        ,  92950.66666667,  62703.33333333, 452482.98      ,
       176150.66666667, 115285.33333333, 216250.66666667,  64276.66666667,
       133688.        , 130791.34      ])

In [ ]:
labels = housing_labels.iloc[some_data.index]
labels

5939     155500.0
2203      91300.0
16297     56400.0
4653     433800.0
1813     167800.0
4989     116400.0
11277    231300.0
16271     65900.0
13964    132000.0
12180    129200.0
Name: median_house_value, dtype: float64

In [ ]:
#cek sekali lagi erornya berapa
lin_mse = mean_squared_error(labels, pred)
trainScore = np.around(np.sqrt(lin_mse), decimals=1)
trainScore

9086.2

In [ ]:
grid = -grid_search.best_score_
np.around(np.sqrt(grid), decimals=1)

48753.5

Simpan Model untuk Proyek Selanjutnya

In [ ]:
import joblib

pipelineReg = full_pipeline_with_predictor
joblib.dump(pipelineReg, "pipeReg.pkl")

!cp pipeReg.pkl /content/drive/MyDrive

In [ ]:
#Load model
my_model_loaded = joblib.load("/content/drive/MyDrive/pipeReg.pkl")

In [ ]:
# housing1 = raw
# housing1

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.325,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.301,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.257,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.643,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.846,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.560,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.557,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.700,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.867,84700.0,INLAND


In [ ]:
housing_ = housing.copy()

In [ ]:
housing_labels = housing_['median_house_value']
housing_ = housing_.drop(columns = 'median_house_value')

In [ ]:
my_model_loaded.fit(housing_, housing_labels)
my_model_loaded.predict(some_data)

array([166840.        ,  92950.66666667,  62703.33333333, 452482.98      ,
       176150.66666667, 115285.33333333, 216250.66666667,  64276.66666667,
       133688.        , 130791.34      ])

In [ ]:
full_pipeline_with_predictor.predict(some_data)

array([166840.        ,  92950.66666667,  62703.33333333, 452482.98      ,
       176150.66666667, 115285.33333333, 216250.66666667,  64276.66666667,
       133688.        , 130791.34      ])

Kesimpulannya hasil prediksi sama